# Understanding Categorical Similarity Space

CategoricalSimilaritySpace is best used to represent categorical similarity information where there are few categories which don't have semantic names to embed them as text. The space creates an n-hot encoding of the categories.

In [1]:
%pip install superlinked==8.8.3

In [2]:
import pandas as pd

from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import StringList
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.space.categorical_similarity_space import (
    CategoricalSimilaritySpace,
)
from superlinked.framework.dsl.query.param import Param

from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.query.query import Query

pd.set_option("display.max_colwidth", 100)

In [3]:
@schema
class Product:
    id: IdField
    category: StringList


product = Product()

## Creating a categorical embedding

Decision items:
1. What are the `categories` that will get their own column in the [n-hot](https://stats.stackexchange.com/questions/467633/what-exactly-is-multi-hot-encoding-and-how-is-it-different-from-one-hot) encoding.<br>
    Other will be classified as `other` and will be represented in the last column of the encoding.
1. Should items in the other category be similar to each other? Set `uncategorized_as_category` accordingly.<br>
    If set to `True`, all `other` items are similar to each other, while otherwise they never are (even if the same category is encoded).<br>
    If the intention is to make a category value similar to only the same category items, it should be added to `categories`.
1. There is a possibility to set `negative_filter` to a negative number, so non-matching categories will result in negative similarity<br>
    (contrary to simply not contributing to similarity) therefore setting these results substantially back in the order.

In [4]:
category_space_uncategorized_category = CategoricalSimilaritySpace(
    category_input=product.category,
    categories=["category-1", "category-2", "category-3"],
    uncategorized_as_category=True,
)
product_index = Index(category_space_uncategorized_category)

In [5]:
source: InMemorySource = InMemorySource(product)
executor = InMemoryExecutor(sources=[source], indices=[product_index])
app = executor.run()

In [6]:
source.put(
    [
        {"id": "product-1", "category": "category-1"},
        {"id": "product-2", "category": "category-2"},
        {"id": "product-3", "category": ["category-2", "category-3"]},
        {"id": "product-4", "category": "category-3"},
        {"id": "product-5", "category": "category-4"},
    ]
)

In [7]:
query_uncateg_as_categ = (
    Query(product_index)
    .find(product)
    .similar(category_space_uncategorized_category.category, Param("query_category"))
)

Note below that multi-label instances are less similar than a single category instance to a single category query - but similar nevertheless.

In [8]:
result_other_uncateg_as_categ = app.query(
    query_uncateg_as_categ, query_category="category-2"
)
result_other_uncateg_as_categ.to_pandas()

,category,id
0,category-2,product-2
1,"category-2, category-3",product-3
2,category-1,product-1
3,category-3,product-4
4,category-4,product-5


Let's first see how the space works with `uncategorized_as_category=True`!

In this case, category-4 category items are similar to other category-4 category items.

In [9]:
result_sunglass_uncateg_as_categ = app.query(
    query_uncateg_as_categ, query_category="category-4"
)

result_sunglass_uncateg_as_categ.to_pandas()

,category,id
0,category-4,product-5
1,category-1,product-1
2,category-2,product-2
3,"category-2, category-3",product-3
4,category-3,product-4


But every `other` category item is similar to category-4 category items, too.

In [10]:
result_other_uncateg_as_categ = app.query(
    query_uncateg_as_categ, query_category="any other category"
)

result_other_uncateg_as_categ.to_pandas()

,category,id
0,category-4,product-5
1,category-1,product-1
2,category-2,product-2
3,"category-2, category-3",product-3
4,category-3,product-4


On the contrary, if we se `uncategorized_as_category=False`, no `other` category will be similar to each other.

In [11]:
category_space_no_uncategorized = CategoricalSimilaritySpace(
    category_input=product.category,
    categories=["category-1", "category-2", "category-3"],
    uncategorized_as_category=False,
)
product_index = Index(category_space_no_uncategorized)

In [12]:
source: InMemorySource = InMemorySource(product)
executor = InMemoryExecutor(sources=[source], indices=[product_index])
app = executor.run()

In [13]:
source.put(
    [
        {"id": "product-1", "category": "category-1"},
        {"id": "product-2", "category": "category-2"},
        {"id": "product-3", "category": "category-2"},
        {"id": "product-4", "category": "category-3"},
        {"id": "product-5", "category": "category-4"},
    ]
)

Neither category-4 to other category-4...

In [14]:
query_uncateg_not_categ = (
    Query(product_index)
    .find(product)
    .similar(category_space_no_uncategorized.category, Param("query_category"))
)
result_uncateg_not_categ = app.query(
    query_uncateg_not_categ, query_category="category-4"
)

result_uncateg_not_categ.to_pandas()

,category,id
0,category-1,product-1
1,category-2,product-2
2,category-2,product-3
3,category-3,product-4
4,category-4,product-5


...nor any `other` category to category-4.

In [15]:
result_uncateg_not_categ = app.query(
    query_uncateg_not_categ, query_category="something_else"
)

result_uncateg_not_categ.to_pandas()

,category,id
0,category-1,product-1
1,category-2,product-2
2,category-2,product-3
3,category-3,product-4
4,category-4,product-5
